In [2]:
# zipファイルダウンロード
url = 'https://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip'
zip = '794_ruby_4237.zip'
import urllib.request
urllib.request.urlretrieve(url, zip)

# ダウンロードしたzipの解凍
import zipfile
with zipfile.ZipFile(zip, 'r') as myzip:
    myzip.extractall()
    # 解凍後のファイルからデータ読み込み
    for myfile in myzip.infolist():
        # 解凍後ファイル名取得
        filename = myfile.filename
        # ファイルオープン時にencodingを指定してsjisの変換をする
        with open(filename, encoding='sjis') as file:
            text = file.read()

# ファイル整形
import re
# ヘッダ部分の除去
text = re.split('\-{5,}',text)[2]
# フッタ部分の除去
text = re.split('底本：',text)[0]
# | の除去
text = text.replace('|', '')
# ルビの削除
text = re.sub('《.+?》', '', text)
# 入力注の削除
text = re.sub('［＃.+?］', '',text)
# 空行の削除
text = re.sub('\n\n', '\n', text) 
text = re.sub('\r', '', text)

# 整形結果確認

# 頭の100文字の表示 
print(text[:100])
# 見やすくするため、空行 
print()
print()
# 後ろの100文字の表示 
print(text[-100:])



一
　うとうととして目がさめると女はいつのまにか、隣のじいさんと話を始めている。このじいさんはたしかに前の前の駅から乗ったいなか者である。発車まぎわに頓狂な声を出して駆け込んで来て、いきなり肌をぬい


評に取りかかる。与次郎だけが三四郎のそばへ来た。
「どうだ森の女は」
「森の女という題が悪い」
「じゃ、なんとすればよいんだ」
　三四郎はなんとも答えなかった。ただ口の中で迷羊、迷羊と繰り返した。




In [3]:
# Janomeのインストール
!pip install janome | tail -n 1

# Janomeのロード
from janome.tokenizer import Tokenizer

# Tokenizerインスタンスの生成 
t = Tokenizer()

# テキストを引数として、形態素解析の結果、名詞・動詞・形容詞(原形)のみを配列で抽出する関数を定義 
def extract_words(text):
    tokens = t.tokenize(text)
    return [token.base_form for token in tokens 
        if token.part_of_speech.split(',')[0] in['名詞', '動詞']]

#  関数テスト
# ret = extract_words('三四郎は京都でちょっと用があって降りたついでに。')
# for word in ret:
#    print(word)

# 全体のテキストを句点('。')で区切った配列にする。 
sentences = text.split('。')
# それぞれの文章を単語リストに変換(処理に数分かかります)
word_list = [extract_words(sentence) for sentence in sentences]

# 結果の一部を確認 
for word in word_list[0]:
    print(word)


一
する
目
さめる
女
隣
じいさん
話
始める
いる


In [8]:
# Word2Vecライブラリの導入
!pip install gensim  

# Word2Vecライブラリのロード
from gensim.models import Word2Vec

# size: 圧縮次元数
# min_count: 出現頻度の低いものをカットする
# window: 前後の単語を拾う際の窓の広さを決める
# iter: 機械学習の繰り返し回数(デフォルト:5)十分学習できていないときにこの値を調整する
# model.wv.most_similarの結果が1に近いものばかりで、model.dict['wv']のベクトル値が小さい値ばかりの 
# ときは、学習回数が少ないと考えられます。
# その場合、iterの値を大きくして、再度学習を行います。

# 事前準備したword_listを使ってWord2Vecの学習実施

# Word2Vecの学習
model = Word2Vec(word_list, vector_size=100, min_count=5, window=5, epochs=100)



In [9]:
# 結果の確認1
# 一つ一つの単語は100次元のベクトルになっています。 
# 「世間」のベクトル値を確認します。
print(model.__dict__['wv']['世間'])


[-4.52727340e-02  4.81914937e-01  8.55659023e-02 -5.73488139e-02
 -3.78183007e-01 -5.85473001e-01  1.26435116e-01  2.63076276e-01
 -4.28592384e-01  1.33654878e-01  4.97842520e-01 -5.35521686e-01
  1.66788518e-01  4.53067899e-01  6.66905046e-01  5.82169145e-02
 -4.45041060e-01 -1.23364680e-01  1.28373399e-01 -6.26604259e-01
 -9.47785258e-01  7.07497299e-01  6.34473145e-01 -1.20512748e+00
 -7.45042460e-04 -1.42308213e-02 -6.01802349e-01 -3.01979810e-01
 -1.28457889e-01 -7.01648831e-01  4.38858628e-01 -3.45663249e-01
  2.27053426e-02 -2.99778879e-02 -3.49818677e-01  2.95587540e-01
 -3.97106767e-01 -3.12760234e-01  2.76371777e-01 -4.43479270e-01
  7.30625808e-01  6.73559904e-02 -4.08077478e-01 -5.40031195e-01
 -3.47804219e-01  5.34878612e-01 -3.15442891e-03 -1.83802452e-02
  6.12846613e-01 -5.95347762e-01  7.31194258e-01 -3.10658962e-01
  5.44622302e-01 -4.60803926e-01  3.47923666e-01 -7.40483522e-01
  4.85124588e-01  4.26251777e-02 -2.96418279e-01  2.42930487e-01
 -9.31129575e-01  3.20085

In [10]:
# 結果の確認2
# 関数most_similarを使って「世間」の類似単語を調べます 
ret = model.wv.most_similar(positive=['世間']) 
for item in ret:
    print(item[0], item[1])


喝采 0.558539867401123
聞こえる 0.5464552044868469
社会 0.5095115900039673
堪える 0.5071780681610107
賛成 0.5068718194961548
自己 0.5028268694877625
外国 0.4804230034351349
ヘーゲル 0.4689410328865051
未来 0.45543158054351807
今日 0.4549022912979126


In [12]:
import numpy as np

arr = np.array([1, 2, 3, 4, 5, 4, 3, 2, 1])
ans = arr.argmax()
print(ans)

4
